# Lecture 7 - Valuation in Practices

In this notebook, we build a complete valuation workflow for Tesla and connect it to ESG policy risk.

What you will practice:

- translating business assumptions into forecast tables,
- linking revenue, gross margin, operating expense, and reinvestment,
- building FCFF-based DCF valuation and sensitivity analysis,
- tracing how tradable regulatory credits affect ratios and intrinsic value.


## Concept Review

### 1. Enterprise Value vs Equity Value

- **Enterprise Value (EV)** is the value of the whole operating business.
- **Equity Value** is the value attributable to shareholders.
- In DCF with FCFF, we usually estimate **EV first**, then adjust for debt/cash/other claims if needed.

### 2. FCFF-based DCF

- Forecast operating performance and reinvestment.
- Compute **FCFF** for each forecast year.
- Discount FCFF and terminal value back to present value.
- Valuation is highly sensitive to **discount rate** and **terminal growth rate**.

### 3. WACC / CAPM / Terminal Growth

- CAPM is used to explain the **cost of equity** conceptually.
- Terminal value often dominates total value, so small changes in `WACC` or `g` can create large valuation changes.

### 4. ESG / Carbon-Policy Link to Valuation

- Tradable regulatory credits are a policy-driven revenue stream.
- This is a concrete bridge from **ESG / emissions regulation** to **accounting numbers** and **intrinsic value**.

## Initial Data Preparation

In this stage, we prepare the data used by the valuation model.

Practical workflow for this lesson:

- Load the prepared local datasets (financial statements, market data, and the **single Excel input sheet**).
- Let students edit the **Value** column in one place instead of searching across multiple files.
- Run one Python function to generate the full valuation pipeline and downstream tables.

This makes the model easier to update when a new reporting year is released.

In [1]:
# Setup: local folders, single Excel input sheet, and lightweight helpers
from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import display

DATA_DIR = Path('data')
L7_DATA_DIR = DATA_DIR / 'lecture7_valuation_practices'
INPUT_SHEET_PATH = L7_DATA_DIR / 'tesla_dcf_input_sheet.xlsx'


def dcf_pv(cashflows, discount_rate):
    """
    Present value of a 1..N cashflow series.

    Args:
        cashflows: Forecast cash flows ordered by period (t=1,2,...,N).
        discount_rate: Per-period discount rate in decimal form (e.g., 0.09).
    """
    cashflows = np.asarray(cashflows, dtype=float)
    t = np.arange(1, len(cashflows) + 1)
    return float((cashflows / (1 + discount_rate) ** t).sum())


def terminal_value_growing_perpetuity(fcf_t, discount_rate, g):
    """
    Gordon-growth terminal value at the end of the forecast horizon.

    Args:
        fcf_t: Final forecast-period cash flow.
        discount_rate: Terminal discount rate (decimal).
        g: Terminal growth rate (decimal), must be below discount_rate.
    """
    return float(fcf_t * (1 + g) / (discount_rate - g))


def show_df(df, digits=2):
    """
    Display a DataFrame with a consistent rounding format for teaching output.

    Args:
        df: pandas object supporting `.round(...)` (typically DataFrame/Series).
        digits: Number of decimal places to display.
    """
    display(df.round(digits))


def l7_read_csv(filename, **kwargs):
    """
    Read a CSV file from the lecture 7 data folder.

    Args:
        filename: File name relative to `data/lecture7_valuation_practices`.
        **kwargs: Extra keyword arguments passed to `pandas.read_csv`.
    """
    return pd.read_csv(L7_DATA_DIR / filename, **kwargs)


# Optional notebook-level sensitivity overrides (keep as None to use Excel defaults / custom switch in the sheet)
SENSITIVITY_DISCOUNT_RATES_OVERRIDE = None
SENSITIVITY_GROWTH_RATES_OVERRIDE = None

print('Data folder:', DATA_DIR)
print('Edit this Excel input sheet (Value column only):', INPUT_SHEET_PATH)
print('This notebook defines all valuation functions internally (see the next function cells).')


Data folder: data
Edit this Excel input sheet (Value column only): data\lecture7_valuation_practices\tesla_dcf_input_sheet.xlsx
This notebook defines all valuation functions internally (see the next function cells).


### Integrated Function Library (Optional Reading)

To make this notebook easier to reuse next year, the valuation functions are defined **inside this notebook**.

- Students can ignore the function cells and only update the Excel input sheet.
- Interested students can open these cells to study the implementation details.

In [2]:
# Function Cell 1 (Optional Reading): Minimal Excel input-sheet loader
# Keep only the parsing logic needed by the notebook.
from copy import deepcopy


def _parse_bool_like(value) -> bool:
    if isinstance(value, (bool, np.bool_)):
        return bool(value)
    if isinstance(value, (int, np.integer, float, np.floating)) and not pd.isna(value):
        return bool(int(value))
    return str(value).strip().lower() in {"1", "true", "t", "yes", "y"}


def _parse_sheet_value(value, value_type: str):
    if pd.isna(value):
        return np.nan
    value_type = str(value_type).strip().lower()
    if value_type == "bool":
        return _parse_bool_like(value)
    if value_type == "int":
        return int(float(value))
    if value_type == "float":
        return float(value)
    return str(value)


def _assign_path(container: dict, path: str, value):
    tokens: list[object] = []
    i = 0
    while i < len(path):
        ch = path[i]
        if ch == ".":
            i += 1
            continue
        if ch == "[":
            j = path.index("]", i)
            tokens.append(int(path[i + 1 : j]) - 1)  # sheet uses 1-based indices
            i = j + 1
            continue
        j = i
        while j < len(path) and path[j] not in ".[":
            j += 1
        tokens.append(path[i:j])
        i = j

    cur = container
    for idx, token in enumerate(tokens):
        is_last = idx == len(tokens) - 1
        next_token = None if is_last else tokens[idx + 1]

        if isinstance(token, str):
            if is_last:
                cur[token] = value
                return
            cur = cur.setdefault(token, [] if isinstance(next_token, int) else {})
            continue

        if not isinstance(cur, list):
            raise TypeError(f"Path segment {token} expects a list in path '{path}'")
        while len(cur) <= token:
            cur.append(None)
        if is_last:
            cur[token] = value
            return
        if cur[token] is None:
            cur[token] = [] if isinstance(next_token, int) else {}
        cur = cur[token]


def load_input_sheet_excel(path: str | Path, sheet_name: str = "inputs") -> dict:
    """Load the classroom flat Excel input sheet into nested dictionaries."""
    table = pd.read_excel(Path(path), sheet_name=sheet_name)
    table["Path"] = table["Path"].astype(str).str.strip()
    table = table.loc[table["Path"].ne("")].copy()
    table["ValueType"] = table["ValueType"].astype(str).str.strip().str.lower()

    payload: dict[str, object] = {}
    for row in table.itertuples(index=False):
        _assign_path(payload, row.Path, _parse_sheet_value(row.Value, row.ValueType))

    raw_sensitivity = payload.get("sensitivity")
    sensitivity = deepcopy(raw_sensitivity) if isinstance(raw_sensitivity, dict) else {}
    for key in ["custom_discount_rates", "custom_growth_rates"]:
        arr = sensitivity.get(key)
        if isinstance(arr, list):
            sensitivity[key] = [float(x) for x in arr if x is not None and not pd.isna(x)]
    sensitivity["use_custom_grid"] = _parse_bool_like(sensitivity.get("use_custom_grid", 0))

    extra_sections = {
        k: v for k, v in payload.items() if k not in {"core", "constants", "sensitivity", "ui"}
    }
    return {
        "core": payload.get("core", {}),
        "constants": payload.get("constants", {}),
        "sensitivity": sensitivity,
        "ui": payload.get("ui", {}),
        "extra_sections": extra_sections,
        "payload": payload,
        "input_table": table,
    }


In [3]:
# Function Cell 2 (Optional Reading): Forecast and DCF model builders
# These functions transform the Excel inputs into revenue, margins, opex, reinvestment, FCFF, and valuation outputs.
from typing import Callable

def cagr(start: float, end: float, n_years: int) -> float:
    """
    Compound annual growth rate helper.

    Args:
        start: Starting value (>0 for standard CAGR interpretation).
        end: Ending value.
        n_years: Number of compounding periods/years.
    """
    start = float(start)
    end = float(end)
    return (end / start) ** (1 / n_years) - 1

def build_revenue_forecast_from_core(core: dict) -> tuple[pd.DataFrame, pd.Series]:
    """
    Build the 10-period revenue forecast table from `core` assumptions.

    Args:
        core: Nested core input dictionary; reads `core["revenue"]` assumptions.
    """
    cfg = core["revenue"]
    base = cfg["base_segment_revenue_billion"]
    target = cfg["target_revenue_billion_period_10"]
    periods = np.arange(1, 11, dtype=int)
    auto_growth = np.asarray(cfg["automotive_growth_path_period_1_to_10"], dtype=float)

    auto = []
    prev_auto = float(base["automotive"])
    for g in auto_growth[:9]:
        prev_auto = prev_auto * (1 + g)
        auto.append(prev_auto)
    auto.append(float(target["automotive"]))
    auto = np.asarray(auto, dtype=float)

    energy_cagr = cagr(base["energy"], target["energy"], 10)
    software_cagr = cagr(base["software"], target["software"], 10)
    energy = np.asarray([base["energy"] * (1 + energy_cagr) ** p for p in periods], dtype=float)
    software = np.asarray([base["software"] * (1 + software_cagr) ** p for p in periods], dtype=float)

    robot = np.zeros(10, dtype=float)
    launch_period = int(cfg["robotaxi"]["launch_period"])
    launch_rev = float(cfg["robotaxi"]["launch_revenue_billion"])
    robot_target = float(target["robotaxi"])
    robot_cagr = cagr(launch_rev, robot_target, 6)
    robot[launch_period - 1] = launch_rev
    for idx in range(launch_period, 10):
        robot[idx] = robot[idx - 1] * (1 + robot_cagr)
    robot[-1] = robot_target

    rev_tbl = pd.DataFrame(
        {
            "Forecast Period": periods,
            "Total Revenue": auto + energy + software + robot,
            "Automotive": auto,
            "growth rate": auto_growth,
            "Energy": energy,
            "Software": software,
            "Robotaxi": robot,
        }
    )
    base_row = pd.Series(
        {
            "Forecast Period": f"Base Year ({cfg['base_year']})",
            "Total Revenue": float(sum(base.values())),
            "Automotive": float(base["automotive"]),
            "growth rate": np.nan,
            "Energy": float(base["energy"]),
            "Software": float(base["software"]),
            "Robotaxi": float(base["robotaxi"]),
        }
    )
    return rev_tbl, base_row

def build_gross_margin_from_core(
    core: dict, rev_tbl: pd.DataFrame, software_revenue_mode: str = "workbook_link"
) -> pd.DataFrame:
    """
    Build gross-margin and gross-profit forecast tables.

    Args:
        core: Nested core input dictionary; reads gross-margin and robotaxi assumptions.
        rev_tbl: Revenue forecast table returned by `build_revenue_forecast_from_core`.
        software_revenue_mode: `workbook_link` replicates workbook linkage; `forecast_software` uses software revenue directly.
    """
    gm = core["gross_margin"]
    n = len(rev_tbl)

    # Automotive margin path replicates the workbook's recursive formulas exactly.
    f3 = float(gm["automotive_margin_base"])
    f8 = float(gm["automotive_margin_target"])
    f9 = f8
    f10 = f8
    f4 = f3
    f5 = f4 + (f8 - f3) / 4
    f6 = f5 + (f9 - f4) / 4
    f7 = f6 + (f10 - f5) / 4
    auto_margin = np.array([f4, f5, f6, f7, f8, f8, f8, f8, f8, f8], dtype=float)

    i3 = float(gm["energy_margin_base"])
    i13 = float(gm["energy_margin_target"])
    energy_margin = np.zeros(n, dtype=float)
    energy_margin[0] = i3 + (i13 - i3) / 10
    for i in range(1, 9):
        energy_margin[i] = energy_margin[i - 1] + (i13 - i3) / 9
    energy_margin[9] = i13

    l3 = float(gm["software_margin_base"])
    l13 = float(gm["software_margin_target"])
    software_margin = np.zeros(n, dtype=float)
    software_margin[0] = l3 + (l13 - l3) / 10
    for i in range(1, 9):
        software_margin[i] = software_margin[i - 1] + (l13 - l3) / 10
    software_margin[9] = l13

    o7 = float(gm["robotaxi_margin_launch"])
    o13 = float(gm["robotaxi_margin_target"])
    robot_margin = np.zeros(n, dtype=float)
    launch_period = int(core["revenue"]["robotaxi"]["launch_period"])
    robot_margin[launch_period - 1] = o7
    for i in range(launch_period, 9):
        robot_margin[i] = robot_margin[i - 1] + (o13 - o7) / 6
    robot_margin[9] = o13

    out = pd.DataFrame({"Forecast Period": rev_tbl["Forecast Period"].to_numpy(dtype=int)})
    out["Total Revenue"] = rev_tbl["Total Revenue"].to_numpy(dtype=float)
    out["Auto revenue"] = rev_tbl["Automotive"].to_numpy(dtype=float)
    out["Energy revenue"] = rev_tbl["Energy"].to_numpy(dtype=float)
    out["Software revenue"] = rev_tbl["Software"].to_numpy(dtype=float)
    out["Robotaxi revenue"] = rev_tbl["Robotaxi"].to_numpy(dtype=float)

    out["Auto GM"] = auto_margin
    out["Energy GM"] = energy_margin
    out["Software GM"] = software_margin
    out["Robotaxi GM"] = np.where(out["Robotaxi revenue"].to_numpy(dtype=float) > 0, robot_margin, np.nan)

    if software_revenue_mode == "workbook_link":
        software_rev_used = out["Energy revenue"].to_numpy(dtype=float)
        software_gp_col = "Software GP (workbook logic)"
    elif software_revenue_mode == "forecast_software":
        software_rev_used = out["Software revenue"].to_numpy(dtype=float)
        software_gp_col = "Software GP"
    else:
        raise ValueError("software_revenue_mode must be 'workbook_link' or 'forecast_software'")

    out["Software revenue used for GP"] = software_rev_used
    out["Auto GP"] = out["Auto revenue"] * out["Auto GM"]
    out["Energy GP"] = out["Energy revenue"] * out["Energy GM"]
    out[software_gp_col] = software_rev_used * out["Software GM"]
    out["Robotaxi GP"] = out["Robotaxi revenue"].to_numpy(dtype=float) * np.nan_to_num(
        out["Robotaxi GM"].to_numpy(dtype=float), nan=0.0
    )
    out["Total gross profit"] = out["Auto GP"] + out["Energy GP"] + out[software_gp_col] + out["Robotaxi GP"]
    out["Total gross margin"] = out["Total gross profit"] / out["Total Revenue"]
    return out

def build_operating_expense_from_core(
    core: dict, rev_tbl: pd.DataFrame, gross_margin_tbl: pd.DataFrame
) -> pd.DataFrame:
    """
    Build operating-expense and EBIT forecast outputs.

    Args:
        core: Nested core input dictionary; reads operating-expense assumptions.
        rev_tbl: Revenue forecast table.
        gross_margin_tbl: Gross-margin forecast table used to derive total gross profit.
    """
    cfg = core["operating_expense"]
    rd_growth = np.asarray(cfg["rd_growth_path_period_1_to_10"], dtype=float)
    n = len(rd_growth)

    rd = np.zeros(n, dtype=float)
    prev = float(cfg["rd_base_billion"])
    for i, g in enumerate(rd_growth):
        prev = prev * (1 + g)
        rd[i] = prev

    sga = np.full(n, float(cfg["sga_constant_billion"]), dtype=float)
    other = np.full(n, float(cfg["other_constant_billion"]), dtype=float)
    total_opex = rd + sga + other

    total_gp = gross_margin_tbl["Total gross profit"].to_numpy(dtype=float)
    revenue = rev_tbl["Total Revenue"].to_numpy(dtype=float)
    ebit = total_gp - total_opex
    op_margin = ebit / revenue

    return pd.DataFrame(
        {
            "Forecast Period": rev_tbl["Forecast Period"].to_numpy(dtype=int),
            "Total Revenue": revenue,
            "Total gross profit": total_gp,
            "R&D": rd,
            "R&D growth": rd_growth,
            "SG&A": sga,
            "Restructuring and other": other,
            "Total operating expense": total_opex,
            "Operating margin": op_margin,
            "Operating profit (EBIT)": ebit,
        }
    )

def build_reinvestment_from_core(core: dict, rev_tbl: pd.DataFrame) -> pd.DataFrame:
    """
    Build reinvestment schedule from sales-to-capital assumptions.

    Args:
        core: Nested core input dictionary; reads reinvestment assumptions.
        rev_tbl: Revenue forecast table used to compute period revenue changes.
    """
    stc = np.asarray(core["reinvestment"]["sales_to_capital_path_period_1_to_10"], dtype=float)
    base_total_rev = float(sum(core["revenue"]["base_segment_revenue_billion"].values()))
    revenue = rev_tbl["Total Revenue"].to_numpy(dtype=float)
    delta_revenue = np.r_[revenue[0] - base_total_rev, np.diff(revenue)]
    return pd.DataFrame(
        {
            "Forecast Period": rev_tbl["Forecast Period"].to_numpy(dtype=int),
            "Sales to Capital": stc,
            "Revenue growth": delta_revenue,
            "Reinvestment": delta_revenue / stc,
        }
    )

def build_fcff_from_core(
    core: dict, rev_tbl: pd.DataFrame, opex_tbl: pd.DataFrame, reinvestment_tbl: pd.DataFrame
) -> pd.DataFrame:
    """
    Build free cash flow to firm (FCFF) from forecast tables and tax assumptions.

    Args:
        core: Nested core input dictionary; reads valuation tax rate.
        rev_tbl: Revenue forecast table.
        opex_tbl: Operating-expense / EBIT table.
        reinvestment_tbl: Reinvestment schedule table.
    """
    tax = float(core["valuation"]["tax_rate"])
    revenue = rev_tbl["Total Revenue"].to_numpy(dtype=float)
    op_margin = opex_tbl["Operating margin"].to_numpy(dtype=float)
    ebit = revenue * op_margin
    ebit_after_tax = ebit * (1 - tax)
    reinvestment = reinvestment_tbl["Reinvestment"].to_numpy(dtype=float)
    fcff = ebit_after_tax - reinvestment
    return pd.DataFrame(
        {
            "Forecast Period": rev_tbl["Forecast Period"].to_numpy(dtype=int),
            "Total Revenues": revenue,
            "Operating Margin": op_margin,
            "EBIT": ebit,
            "EBIT(1-t)": ebit_after_tax,
            "Reinvestment": reinvestment,
            "FCFF": fcff,
        }
    )

def build_valuation_from_core(
    core: dict,
    fcff_tbl: pd.DataFrame,
    dcf_pv_func: Callable[[np.ndarray, float], float],
    terminal_value_func: Callable[[float, float, float], float],
) -> dict:
    """
    Compute valuation assumptions summary and intrinsic-value components.

    Args:
        core: Nested core input dictionary; reads valuation assumptions and capital structure inputs.
        fcff_tbl: FCFF forecast table containing the `FCFF` column.
        dcf_pv_func: Present-value function for the forecast cash-flow stream.
        terminal_value_func: Terminal-value function (e.g., growing perpetuity).
    """
    v = core["valuation"]
    rf = float(v["risk_free_rate"])
    erp = float(v["equity_risk_premium"])
    beta = float(v["beta"])
    kd = float(v["cost_of_debt"])
    equity_value = float(v["equity_value_input_billion"])
    debt_value = float(v["debt_value_input_billion"])
    cost_of_equity = rf + beta * erp

    # Replicate the workbook formula exactly (including the workbook's structure).
    workbook_wacc_formula = (cost_of_equity * cost_of_equity) / (equity_value + debt_value) + kd * debt_value / (
        equity_value + debt_value
    )

    w_forecast = float(v["forecast_discount_rate_workbook"])
    w_terminal = float(v["terminal_discount_rate"])
    g = float(v["terminal_growth_rate"])
    fcff = fcff_tbl["FCFF"].to_numpy(dtype=float)

    pv_forecast = dcf_pv_func(fcff, w_forecast)
    terminal_value = terminal_value_func(fcff[-1], w_terminal, g)
    pv_terminal = terminal_value / (1 + w_terminal) ** len(fcff)

    return {
        "assumptions": pd.Series(
            {
                "Tax rate": float(v["tax_rate"]),
                "Risk-free rate": rf,
                "Equity risk premium": erp,
                "Beta": beta,
                "Cost of equity (CAPM)": cost_of_equity,
                "Cost of debt": kd,
                "Equity value input (bn)": equity_value,
                "Debt value input (bn)": debt_value,
                "Forecast discount rate (workbook)": w_forecast,
                "Terminal discount rate": w_terminal,
                "Terminal growth rate": g,
                "Workbook WACC formula result": workbook_wacc_formula,
            }
        ),
        "intrinsic_value_components": pd.Series(
            {
                "Terminal Value": terminal_value,
                "PV(Terminal Value)": pv_terminal,
                "PV(CF over Forecast Period)": pv_forecast,
                "Value of Operating Assets": pv_forecast + pv_terminal,
            }
        ),
    }

def build_workbook_style_model(
    core: dict,
    dcf_pv_func: Callable[[np.ndarray, float], float],
    terminal_value_func: Callable[[float, float, float], float],
    software_revenue_mode: str = "workbook_link",
) -> dict:
    """
    Run the full workbook-style forecasting and valuation pipeline.

    Args:
        core: Nested core model input dictionary.
        dcf_pv_func: Function used to discount the explicit forecast-period FCFF.
        terminal_value_func: Function used to compute terminal value.
        software_revenue_mode: Gross-profit linkage rule for the software line item.
    """
    rev_tbl, base_row = build_revenue_forecast_from_core(core)
    gross_margin_tbl = build_gross_margin_from_core(core, rev_tbl, software_revenue_mode=software_revenue_mode)
    opex_tbl = build_operating_expense_from_core(core, rev_tbl, gross_margin_tbl)
    reinvestment_tbl = build_reinvestment_from_core(core, rev_tbl)
    fcff_tbl = build_fcff_from_core(core, rev_tbl, opex_tbl, reinvestment_tbl)
    valuation = build_valuation_from_core(core, fcff_tbl, dcf_pv_func, terminal_value_func)
    return {
        "base_row": base_row,
        "revenue": rev_tbl,
        "gross_margin": gross_margin_tbl,
        "operating_expense": opex_tbl,
        "reinvestment": reinvestment_tbl,
        "fcff": fcff_tbl,
        "valuation": valuation,
    }

In [4]:
# Function Cell 3 (Optional Reading): Sensitivity analysis and one-call valuation pipeline
# This cell provides the integrated function used later for result generation.

def build_intrinsic_value_sensitivity_matrix(
    fcff: np.ndarray | pd.Series,
    *,
    dcf_pv_func: Callable[[np.ndarray, float], float],
    terminal_value_func: Callable[[float, float, float], float],
    discount_rates: list[float] | np.ndarray,
    growth_rates: list[float] | np.ndarray,
) -> pd.DataFrame:
    """
    Build an intrinsic-value sensitivity matrix over discount/growth assumptions.

    Args:
        fcff: Forecast FCFF series (array-like).
        dcf_pv_func: Present-value function used for forecast-period FCFF.
        terminal_value_func: Terminal-value function used for the final-period FCFF.
        discount_rates: Row grid of discount rates (decimals).
        growth_rates: Column grid of terminal growth rates (decimals).
    """
    fcff_arr = np.asarray(fcff, dtype=float)
    out = pd.DataFrame(
        index=[f"{float(r):.2%}" for r in discount_rates],
        columns=[f"{float(g):.2%}" for g in growth_rates],
        dtype=float,
    )
    for r_ in discount_rates:
        for g_ in growth_rates:
            r_ = float(r_)
            g_ = float(g_)
            pv_f = dcf_pv_func(fcff_arr, r_)
            tv_ = terminal_value_func(fcff_arr[-1], r_, g_)
            pv_tv_ = tv_ / (1 + r_) ** len(fcff_arr)
            out.loc[f"{r_:.2%}", f"{g_:.2%}"] = pv_f + pv_tv_
    return out

def _coerce_override_grid(override, default_value: float) -> list[float]:
    """
    Normalize optional notebook-level override arrays to a float list.

    Args:
        override: Optional array-like override. If None, the default value is used as a single-point grid.
        default_value: Fallback scalar when `override` is None.
    """
    values = [default_value] if override is None else override
    return [float(x) for x in values]

def build_sensitivity_grids(
    core: dict,
    sensitivity_cfg: dict | None = None,
    *,
    discount_rates_override: list[float] | np.ndarray | None = None,
    growth_rates_override: list[float] | np.ndarray | None = None,
) -> tuple[list[float], list[float]]:
    """
    Build the active sensitivity grids for discount and terminal-growth assumptions.

    Args:
        core: Nested core input dictionary; reads terminal discount/growth rates.
        sensitivity_cfg: Optional sensitivity config from the Excel input sheet.
        discount_rates_override: Optional notebook-level discount-rate grid override.
        growth_rates_override: Optional notebook-level terminal-growth grid override.

    Default behavior uses terminal discount rate and terminal growth rate from the valuation inputs.
    If custom mode is enabled in the input sheet (or explicit overrides are passed), use custom arrays.
    """
    sensitivity_cfg = deepcopy(sensitivity_cfg or {})

    if discount_rates_override is not None or growth_rates_override is not None:
        return (
            _coerce_override_grid(discount_rates_override, float(core["valuation"]["terminal_discount_rate"])),
            _coerce_override_grid(growth_rates_override, float(core["valuation"]["terminal_growth_rate"])),
        )

    use_custom = bool(sensitivity_cfg.get("use_custom_grid", False))
    center_r = float(core["valuation"]["terminal_discount_rate"])
    center_g = float(core["valuation"]["terminal_growth_rate"])

    r_vals = [float(x) for x in sensitivity_cfg.get("custom_discount_rates", [])]
    g_vals = [float(x) for x in sensitivity_cfg.get("custom_growth_rates", [])]
    if use_custom and r_vals and g_vals:
        return r_vals, g_vals

    dl = float(sensitivity_cfg.get("default_discount_delta_large", 0.01))
    ds = float(sensitivity_cfg.get("default_discount_delta_small", 0.005))
    dg = float(sensitivity_cfg.get("default_growth_delta", 0.005))
    discount_rates = [center_r - dl, center_r - ds, center_r, center_r + ds, center_r + dl]
    growth_rates = [center_g - dg, center_g, center_g + dg]
    return discount_rates, growth_rates

def run_valuation_from_input_sheet(
    input_sheet_path: str | Path,
    *,
    dcf_pv_func: Callable[[np.ndarray, float], float],
    terminal_value_func: Callable[[float, float, float], float],
    sheet_name: str = "inputs",
    software_revenue_mode: str = "workbook_link",
    discount_rates_override: list[float] | np.ndarray | None = None,
    growth_rates_override: list[float] | np.ndarray | None = None,
) -> dict:
    """
    Load Excel inputs, run the valuation model, and return all derived outputs.

    Args:
        input_sheet_path: Path to the classroom Excel input file.
        dcf_pv_func: Present-value function for forecast-period FCFF.
        terminal_value_func: Terminal-value function for the last forecast-period FCFF.
        sheet_name: Worksheet name for the flat input table.
        software_revenue_mode: Gross-profit linkage rule for the software segment.
        discount_rates_override: Optional notebook-level discount-rate grid override.
        growth_rates_override: Optional notebook-level terminal-growth grid override.
    """
    package = load_input_sheet_excel(input_sheet_path, sheet_name=sheet_name)
    core = package["core"]
    model = build_workbook_style_model(
        core,
        dcf_pv_func=dcf_pv_func,
        terminal_value_func=terminal_value_func,
        software_revenue_mode=software_revenue_mode,
    )
    discount_rates, growth_rates = build_sensitivity_grids(
        core,
        package.get("sensitivity", {}),
        discount_rates_override=discount_rates_override,
        growth_rates_override=growth_rates_override,
    )
    sensitivity_matrix = build_intrinsic_value_sensitivity_matrix(
        model["fcff"]["FCFF"].to_numpy(dtype=float),
        dcf_pv_func=dcf_pv_func,
        terminal_value_func=terminal_value_func,
        discount_rates=discount_rates,
        growth_rates=growth_rates,
    )
    return {
        "core": core,
        "constants": package.get("constants", {}),
        "sensitivity_config": package.get("sensitivity", {}),
        "ui": package.get("ui", {}),
        "extra_sections": package.get("extra_sections", {}),
        "payload": package.get("payload", {}),
        "input_table": package.get("input_table"),
        "model": model,
        "sensitivity_matrix": sensitivity_matrix,
        "sensitivity_discount_rates": discount_rates,
        "sensitivity_growth_rates": growth_rates,
    }



def run_integrated_valuation(input_sheet_path, discount_rates_override=None, growth_rates_override=None):
    """
    Student-friendly wrapper: one function call from Excel inputs to full valuation output.

    Args:
        input_sheet_path: Path to the classroom Excel input sheet.
        discount_rates_override: Optional discount-rate grid override for sensitivity output.
        growth_rates_override: Optional terminal-growth grid override for sensitivity output.
    """
    return run_valuation_from_input_sheet(
        input_sheet_path,
        dcf_pv_func=dcf_pv,
        terminal_value_func=terminal_value_growing_perpetuity,
        discount_rates_override=discount_rates_override,
        growth_rates_override=growth_rates_override,
    )

# Teaching module wrappers for concise downstream cells
def prepare_revenue_module_package(core: dict) -> dict:
    """
    Run the revenue module and prepare teaching-friendly display tables.

    Args:
        core: Nested core input dictionary used by the revenue forecast module.
    """
    rev_tbl_full, base_row = build_revenue_forecast_from_core(core)

    base_rev = {
        "Total Revenue": float(base_row["Total Revenue"]),
        "Automotive": float(base_row["Automotive"]),
        "Energy": float(base_row["Energy"]),
        "Software": float(base_row["Software"]),
        "Robotaxi": float(base_row["Robotaxi"]),
    }
    base_total_rev = base_rev["Total Revenue"]

    revenue_input_view = pd.DataFrame(
        {
            "Base (bn)": pd.Series(core["revenue"]["base_segment_revenue_billion"]),
            "Target (period 10, bn)": pd.Series(core["revenue"]["target_revenue_billion_period_10"]),
        }
    )
    revenue_growth_input_view = pd.DataFrame(
        {
            "Automotive growth (input)": core["revenue"]["automotive_growth_path_period_1_to_10"],
        },
        index=[f"Period {i}" for i in range(1, 11)],
    )
    robotaxi_input_info = pd.Series(
        {
            "Robotaxi launch period": int(core["revenue"]["robotaxi"]["launch_period"]),
            "Robotaxi launch revenue (bn)": float(core["revenue"]["robotaxi"]["launch_revenue_billion"]),
        },
        name="Value",
    )

    rev_tbl_display = rev_tbl_full.rename(
        columns={"Total Revenue": "Total Revenue (Billion)", "growth rate": "Automotive growth"}
    ).copy()
    rev_tbl_display[["Total Revenue (Billion)", "Automotive", "Energy", "Software", "Robotaxi"]] = (
        rev_tbl_display[["Total Revenue (Billion)", "Automotive", "Energy", "Software", "Robotaxi"]].round(2)
    )
    rev_tbl_display["Automotive growth"] = rev_tbl_display["Automotive growth"].map(lambda x: f"{x:.2%}")

    base_row_display = pd.DataFrame(
        [
            {
                "Forecast Period": base_row["Forecast Period"],
                "Total Revenue (Billion)": base_rev["Total Revenue"],
                "Automotive": base_rev["Automotive"],
                "Automotive growth": "",
                "Energy": base_rev["Energy"],
                "Software": base_rev["Software"],
                "Robotaxi": base_rev["Robotaxi"],
            }
        ]
    )
    revenue_display = pd.concat([base_row_display, rev_tbl_display], ignore_index=True)

    cagrs = pd.Series(
        {
            "Automotive (10y)": (rev_tbl_full.iloc[-1]["Automotive"] / base_rev["Automotive"]) ** (1 / 10) - 1,
            "Energy (10y)": (rev_tbl_full.iloc[-1]["Energy"] / base_rev["Energy"]) ** (1 / 10) - 1,
            "Software (10y)": (rev_tbl_full.iloc[-1]["Software"] / base_rev["Software"]) ** (1 / 10) - 1,
            "Robotaxi (6y, p4->p10)": (rev_tbl_full.iloc[-1]["Robotaxi"] / rev_tbl_full.loc[3, "Robotaxi"]) ** (1 / 6) - 1,
        },
        name="Implied CAGR",
    )

    return {
        "revenue": rev_tbl_full,
        "base_row": base_row,
        "base_rev": base_rev,
        "base_total_rev": base_total_rev,
        "revenue_input_view": revenue_input_view,
        "revenue_growth_input_view": revenue_growth_input_view,
        "robotaxi_input_info": robotaxi_input_info,
        "revenue_display": revenue_display,
        "cagrs": cagrs,
    }

def prepare_gross_margin_module_package(
    core: dict,
    rev_tbl: pd.DataFrame,
    *,
    software_revenue_mode: str = "workbook_link",
) -> dict:
    """
    Run the gross-margin module and prepare teaching-friendly display tables.

    Args:
        core: Nested core input dictionary.
        rev_tbl: Revenue forecast table returned by the revenue module.
        software_revenue_mode: Gross-profit linkage rule for the software line item.
    """
    gross_margin_tbl = build_gross_margin_from_core(core, rev_tbl, software_revenue_mode=software_revenue_mode)

    margin_input_view = pd.DataFrame(
        {
            "Base": {
                "Automotive": core["gross_margin"]["automotive_margin_base"],
                "Energy": core["gross_margin"]["energy_margin_base"],
                "Software": core["gross_margin"]["software_margin_base"],
                "Robotaxi": core["gross_margin"]["robotaxi_margin_launch"],
            },
            "Target": {
                "Automotive": core["gross_margin"]["automotive_margin_target"],
                "Energy": core["gross_margin"]["energy_margin_target"],
                "Software": core["gross_margin"]["software_margin_target"],
                "Robotaxi": core["gross_margin"]["robotaxi_margin_target"],
            },
        }
    )
    gross_margin_input_notes = pd.Series(
        {
            "Displayed base total gross margin": float(core["gross_margin"]["base_total_gross_margin_display"]),
            "Robotaxi launch period": int(core["revenue"]["robotaxi"]["launch_period"]),
            "Software GP mode": software_revenue_mode,
        },
        name="Value",
    )

    margin_view = gross_margin_tbl[
        ["Forecast Period", "Auto GM", "Energy GM", "Software GM", "Robotaxi GM", "Total gross margin"]
    ].rename(columns={"Robotaxi GM": "Robotaxi GM (from launch onward)"})

    profit_view = gross_margin_tbl[
        [
            "Forecast Period",
            "Auto GP",
            "Energy GP",
            "Software GP (workbook logic)",
            "Robotaxi GP",
            "Total gross profit",
        ]
    ].rename(columns={"Software GP (workbook logic)": "Software GP"})

    return {
        "gross_margin": gross_margin_tbl,
        "margin_input_view": margin_input_view,
        "gross_margin_input_notes": gross_margin_input_notes,
        "margin_view": margin_view,
        "profit_view": profit_view,
    }

def prepare_opex_module_package(core: dict, rev_tbl: pd.DataFrame, gross_margin_tbl: pd.DataFrame) -> dict:
    """
    Run the opex module and prepare teaching-friendly display tables.

    Args:
        core: Nested core input dictionary.
        rev_tbl: Revenue forecast table.
        gross_margin_tbl: Gross-margin forecast table.
    """
    opex_tbl = build_operating_expense_from_core(core, rev_tbl, gross_margin_tbl)

    opex_input_view = pd.Series(
        {
            "R&D base (bn)": float(core["operating_expense"]["rd_base_billion"]),
            "SG&A constant (bn)": float(core["operating_expense"]["sga_constant_billion"]),
            "Restructuring and other constant (bn)": float(core["operating_expense"]["other_constant_billion"]),
        },
        name="Value",
    )
    rd_growth_input_view = pd.DataFrame(
        {"R&D growth (input)": core["operating_expense"]["rd_growth_path_period_1_to_10"]},
        index=[f"Period {i}" for i in range(1, 11)],
    )

    opex_view = opex_tbl[
        [
            "Forecast Period",
            "R&D",
            "R&D growth",
            "SG&A",
            "Restructuring and other",
            "Total operating expense",
            "Operating margin",
            "Operating profit (EBIT)",
        ]
    ].copy()
    return {
        "operating_expense": opex_tbl,
        "opex_input_view": opex_input_view,
        "rd_growth_input_view": rd_growth_input_view,
        "opex_view": opex_view,
    }

def prepare_reinvestment_fcff_module_package(
    core: dict,
    rev_tbl: pd.DataFrame,
    opex_tbl: pd.DataFrame,
) -> dict:
    """
    Run the reinvestment/FCFF module and prepare teaching-friendly display tables.

    Args:
        core: Nested core input dictionary.
        rev_tbl: Revenue forecast table.
        opex_tbl: Operating-expense / EBIT table.
    """
    reinvestment_tbl = build_reinvestment_from_core(core, rev_tbl)
    fcff_tbl = build_fcff_from_core(core, rev_tbl, opex_tbl, reinvestment_tbl)

    reinvestment_input_view = pd.DataFrame(
        {"Sales-to-capital (input)": core["reinvestment"]["sales_to_capital_path_period_1_to_10"]},
        index=[f"Period {i}" for i in range(1, 11)],
    )
    fcff_input_view = pd.DataFrame(
        {
            "Forecast Period": rev_tbl["Forecast Period"],
            "Revenue input to FCFF": rev_tbl["Total Revenue"],
            "Operating margin input to FCFF": opex_tbl["Operating margin"],
            "Tax rate (input)": float(core["valuation"]["tax_rate"]),
        }
    )

    fcff_view = fcff_tbl[
        ["Forecast Period", "Total Revenues", "Operating Margin", "EBIT(1-t)", "Reinvestment", "FCFF"]
    ]
    return {
        "reinvestment": reinvestment_tbl,
        "fcff": fcff_tbl,
        "reinvestment_input_view": reinvestment_input_view,
        "fcff_input_view": fcff_input_view,
        "fcff_view": fcff_view,
    }

def prepare_valuation_sensitivity_module_package(
    core: dict,
    fcff_tbl: pd.DataFrame,
    *,
    dcf_pv_func,
    terminal_value_func,
    sensitivity_cfg: dict | None = None,
    display_inputs: dict | None = None,
    summary_core: dict | None = None,
    discount_rates_override: list[float] | np.ndarray | None = None,
    growth_rates_override: list[float] | np.ndarray | None = None,
) -> dict:
    """
    Run the valuation/sensitivity module and prepare teaching-friendly display tables.

    Args:
        core: Nested core input dictionary used for valuation calculations.
        fcff_tbl: FCFF forecast table containing the `FCFF` column.
        dcf_pv_func: Present-value function for forecast-period FCFF.
        terminal_value_func: Terminal-value function for the final forecast-period FCFF.
        sensitivity_cfg: Optional sensitivity configuration dictionary.
        display_inputs: Optional display-only inputs loaded from the Excel sheet.
        summary_core: Optional core dictionary used for valuation summary labels (defaults to `core`).
        discount_rates_override: Optional notebook-level discount-rate grid override.
        growth_rates_override: Optional notebook-level terminal-growth grid override.
    """
    sensitivity_cfg = sensitivity_cfg or {}
    display_inputs = display_inputs or {}
    summary_core = summary_core or core

    valuation = build_valuation_from_core(
        core,
        fcff_tbl,
        dcf_pv_func=dcf_pv_func,
        terminal_value_func=terminal_value_func,
    )

    sensitivity_discount_rates, sensitivity_growth_rates = build_sensitivity_grids(
        core,
        sensitivity_cfg,
        discount_rates_override=discount_rates_override,
        growth_rates_override=growth_rates_override,
    )
    sensitivity_matrix = build_intrinsic_value_sensitivity_matrix(
        fcff_tbl["FCFF"].to_numpy(dtype=float),
        dcf_pv_func=dcf_pv_func,
        terminal_value_func=terminal_value_func,
        discount_rates=sensitivity_discount_rates,
        growth_rates=sensitivity_growth_rates,
    )

    valuation_input_view = pd.Series(
        {
            "Tax rate": float(summary_core["valuation"]["tax_rate"]),
            "Risk-free rate": float(summary_core["valuation"]["risk_free_rate"]),
            "Equity risk premium": float(summary_core["valuation"]["equity_risk_premium"]),
            "Beta": float(summary_core["valuation"]["beta"]),
            "Cost of debt": float(summary_core["valuation"]["cost_of_debt"]),
            "Equity value input (bn)": float(summary_core["valuation"]["equity_value_input_billion"]),
            "Debt value input (bn)": float(summary_core["valuation"]["debt_value_input_billion"]),
            "Forecast discount rate": float(summary_core["valuation"]["forecast_discount_rate_workbook"]),
            "Terminal discount rate": float(summary_core["valuation"]["terminal_discount_rate"]),
            "Terminal growth rate": float(summary_core["valuation"]["terminal_growth_rate"]),
        },
        name="Value",
    )
    fcff_input_view = fcff_tbl[["Forecast Period", "FCFF"]].copy()

    val_display = display_inputs.get("valuation_display", {})
    beta_est = float(pd.to_numeric(val_display.get("estimated_beta_from_local_daily_data"), errors="coerce"))
    rf_latest = float(pd.to_numeric(val_display.get("latest_local_10y_treasury"), errors="coerce"))
    roe_2025 = float(pd.to_numeric(val_display.get("roe_fy2025"), errors="coerce"))

    valuation_summary = pd.Series(
        {
            "Estimated beta (from Excel input sheet)": beta_est,
            "Latest local 10Y Treasury (from Excel input sheet)": rf_latest,
            "ROE FY2025 (from Excel input sheet)": roe_2025,
            "Tax rate (model input)": float(summary_core["valuation"]["tax_rate"]),
            "Forecast discount rate": float(summary_core["valuation"]["forecast_discount_rate_workbook"]),
            "Terminal discount rate": float(summary_core["valuation"]["terminal_discount_rate"]),
            "Terminal growth rate": float(summary_core["valuation"]["terminal_growth_rate"]),
        },
        name="Value",
    )

    sensitivity_info = pd.Series(
        {
            "Use custom grid from Excel": int(bool(sensitivity_cfg.get("use_custom_grid", False))),
            "Default discount delta (large)": float(sensitivity_cfg.get("default_discount_delta_large", 0.01)),
            "Default discount delta (small)": float(sensitivity_cfg.get("default_discount_delta_small", 0.005)),
            "Default growth delta": float(sensitivity_cfg.get("default_growth_delta", 0.005)),
            "Notebook discount override active": int(discount_rates_override is not None),
            "Notebook growth override active": int(growth_rates_override is not None),
        },
        name="Value",
    )

    active_grids = pd.DataFrame(
        {
            "Discount Rate Grid": pd.Series(sensitivity_discount_rates),
            "Terminal Growth Grid": pd.Series(sensitivity_growth_rates),
        }
    )

    return {
        "valuation": valuation,
        "iv_from_core": valuation["intrinsic_value_components"],
        "valuation_input_view": valuation_input_view,
        "fcff_input_view": fcff_input_view,
        "valuation_summary": valuation_summary,
        "sensitivity_discount_rates": sensitivity_discount_rates,
        "sensitivity_growth_rates": sensitivity_growth_rates,
        "sensitivity_matrix": sensitivity_matrix,
        "sensitivity_info": sensitivity_info,
        "active_grids": active_grids,
        "iv_matrix": sensitivity_matrix.copy(),
    }



### Step 1. Input Parsing and Assumption Snapshot

This section prints the **single Excel input sheet** used by the valuation model.

Students only need to update the **Value** column in the Excel file (keep the **Path** column unchanged).

The Python functions then:

- parse the sheet into a structured model input package,
- build all forecast tables automatically,
- run DCF valuation,
- generate sensitivity analysis using default or custom grids.

In [5]:
# Input parsing cell: Excel input sheet -> structured inputs (no model run yet)
l7_input_package = load_input_sheet_excel(INPUT_SHEET_PATH)

# Refresh notebook-level objects from the latest Excel inputs
L7_CORE_INPUTS = l7_input_package['core']
L7_CONSTANTS = l7_input_package.get('constants', {})
L7_EXTRA_SECTIONS = l7_input_package.get('extra_sections', {})
L7_DISPLAY_INPUTS = L7_EXTRA_SECTIONS.get('display_inputs', {})
L7_UI = l7_input_package.get('ui', {})
L7_SENSITIVITY_CONFIG = l7_input_package.get('sensitivity', {})
L7_INPUT_TABLE = l7_input_package.get('input_table')
core = L7_CORE_INPUTS

# Downstream module cells will populate these step by step.
l7_model = {}
l7_run = {}

print('Input sheet parsed successfully. Update the Excel sheet and rerun this cell to refresh the inputs.')
print('Company:', L7_UI.get('company_name', 'Company'))
print('Module-specific key inputs will be shown inside each module cell (Revenue / GM / Opex / FCFF / Valuation).')

input_section_summary = (
    L7_INPUT_TABLE.groupby('Section', dropna=False)
    .agg(Rows=('Path', 'count'))
    .sort_values('Rows', ascending=False)
)
show_df(input_section_summary, 0)

# Raw input table (single source of truth for the notebook)
display(
    L7_INPUT_TABLE[['Section', 'Path', 'Value', 'Unit', 'Description']]
    .reset_index(drop=True)
)


Input sheet loaded successfully. Update the Excel sheet and rerun this cell to refresh the model.
Company: Tesla


,Section,Path,Value,Unit,Description
0,ui,ui.company_name,Tesla,NaN,ui -> company name
1,ui,ui.notes,Edit the Value column only. Keep Path unchanged.,NaN,ui -> notes
2,core,core.gross_margin.automotive_margin_base,0.178,decimal,gross margin -> automotive margin base
3,core,core.gross_margin.automotive_margin_target,0.2,decimal,gross margin -> automotive margin target
4,core,core.gross_margin.base_total_gross_margin_display,0.18,decimal,gross margin -> base total gross margin display
...,...,...,...,...,...
114,display_inputs,display_inputs.tradable_credit_ratio_inputs_us...,2024,NaN,display inputs -> tradable credit ratio inputs...
115,display_inputs,display_inputs.tradable_credit_ratio_inputs_us...,94827000000,NaN,display inputs -> tradable credit ratio inputs...
116,display_inputs,display_inputs.valuation_display.estimated_bet...,2.052545,decimal,display inputs -> valuation display -> estimat...
117,display_inputs,display_inputs.valuation_display.latest_local_...,0.0418,NaN,display inputs -> valuation display -> latest ...


,Base (bn),"Target (period 10, bn)"
automotive,69.53,300.0
energy,12.77,80.0
robotaxi,0.00,80.0
software,12.53,40.0
robotaxi_launch_revenue,NaN,11.4


,Automotive growth,Sales-to-capital
Period 1,0.25,5.0
Period 2,0.24,5.0
Period 3,0.23,4.0
Period 4,0.22,4.0
Period 5,0.20,3.0
Period 6,0.16,2.0
Period 7,0.12,2.0
Period 8,0.08,2.0
Period 9,0.08,2.0
Period 10,0.08,2.0


,Base,Target
Automotive,0.178,0.20
Energy,0.298,0.38
Software,0.075,0.22
Robotaxi,0.050,0.25


,Value
Tax rate,0.2400
Risk-free rate,0.0408
Equity risk premium,0.1200
Beta,2.0500
Cost of debt,0.0500
Forecast discount rate,0.0929
Terminal discount rate,0.0835
Terminal growth rate,0.0422
"Tradable credit revenue (2025, bn)",1.9330
Sensitivity use_custom_grid (0/1),0.0000


### Reference: Historical Financial Performance

This code summarizes the key labeled historical changes shown on the slide:

- revenue mix shift (Automotive vs Energy / Services)
- operating expense increase (especially R&D)

These observations motivate later forecasting assumptions.

In [6]:
# Historical performance labels loaded from the Excel input sheet (single-source input design)
hist = L7_DISPLAY_INPUTS['historical_performance']
seg = hist['segment_revenue_billion']
opex = hist['operating_expense_billion']

segment_rev = pd.DataFrame({
    '2023': {
        'Automotive': float(seg['automotive_2023']),
        'Energy and Storage': float(seg['energy_and_storage_2023']),
        'Services and Other': float(seg['services_and_other_2023']),
    },
    '2025': {
        'Automotive': float(seg['automotive_2025']),
        'Energy and Storage': float(seg['energy_and_storage_2025']),
        'Services and Other': float(seg['services_and_other_2025']),
    },
})
segment_rev['Change (B)'] = segment_rev['2025'] - segment_rev['2023']
segment_rev['Growth (%)'] = (segment_rev['2025'] / segment_rev['2023'] - 1) * 100
show_df(segment_rev, 2)

opex_labeled = pd.DataFrame({
    '2023': {
        'R&D': float(opex['rd_2023']),
        'SG&A': float(opex['sga_2023']),
        'Other Opex': pd.to_numeric(opex['other_opex_2023'], errors='coerce'),
    },
    '2025': {
        'R&D': float(opex['rd_2025']),
        'SG&A': float(opex['sga_2025']),
        'Other Opex': pd.to_numeric(opex['other_opex_2025'], errors='coerce'),
    },
})
opex_labeled['Change (B)'] = opex_labeled['2025'] - opex_labeled['2023']
opex_labeled['Growth (%)'] = (opex_labeled['2025'] / opex_labeled['2023'] - 1) * 100
show_df(opex_labeled, 2)

,2023,2025,Change (B),Growth (%)
Automotive,82.42,69.53,-12.89,-15.64
Energy and Storage,6.04,12.77,6.73,111.42
Services and Other,8.32,12.53,4.21,50.60


,2023,2025,Change (B),Growth (%)
R&D,3.97,6.41,2.44,61.46
SG&A,4.80,5.83,1.03,21.46
Other Opex,NaN,0.50,NaN,NaN


## Model Building

### Step 2. Revenue Module Run and Display

Teaching objective:

- translate qualitative business assumptions into a structured forecast table,
- separate **assumptions** from **mechanical calculations**, and
- understand how target-anchored rows affect implied growth rates.


In [7]:
# Revenue module run + display
revenue_module = prepare_revenue_module_package(core)
rev_tbl_full = revenue_module['revenue']
base_row = revenue_module['base_row']
base_rev = revenue_module['base_rev']
base_total_rev = revenue_module['base_total_rev']
l7_model.update({'base_row': base_row.copy(), 'revenue': rev_tbl_full.copy()})

print('Revenue module: key inputs used in the forecast')
show_df(revenue_module['revenue_input_view'], 4)
show_df(revenue_module['revenue_growth_input_view'], 4)
show_df(revenue_module['robotaxi_input_info'].to_frame(), 4)

print('Revenue module: forecast outputs')
show_df(revenue_module['revenue_display'], 2)
show_df(revenue_module['cagrs'].to_frame(), 4)


,Forecast Period,Total Revenue (Billion),Automotive,Automotive growth,Energy,Software,Robotaxi
0,Base Year (2025),94.83,69.53,,12.77,12.53,0.00
1,1,116.33,86.91,25.00%,15.34,14.07,0.00
2,2,142.01,107.77,24.00%,18.43,15.80,0.00
3,3,172.45,132.56,23.00%,22.14,17.75,0.00
4,4,219.66,161.72,22.00%,26.60,19.93,11.40
5,5,264.19,194.07,20.00%,31.96,22.39,15.77
6,6,310.49,225.12,16.00%,38.40,25.14,21.83
7,7,356.70,252.13,12.00%,46.13,28.24,30.20
8,8,401.23,272.30,8.00%,55.43,31.71,41.79
9,9,454.11,294.09,8.00%,66.59,35.62,57.82


,Implied CAGR
Automotive (10y),0.1574
Energy (10y),0.2014
Software (10y),0.1231
"Robotaxi (6y, p4->p10)",0.3837


### Step 3. Gross Margin Module Run and Display

This section builds the gross margin model directly in Python.

Teaching focus:

- compute segment-level gross profit from forecast revenues,
- apply margin paths by segment (Automotive / Energy / Software / Robotaxi),
- aggregate to total gross profit and total gross margin.

Implementation note:

- The provided teaching data uses a linked-revenue assumption for the software gross-profit line.  
- We keep that assumption in code so the calculations remain consistent across sections.


In [8]:
# Gross Margin module run + display
gross_margin_module = prepare_gross_margin_module_package(core, rev_tbl_full)
gross_margin_sheet_py = gross_margin_module['gross_margin']
l7_model['gross_margin'] = gross_margin_sheet_py.copy()

print('Gross margin module: key inputs used in the calculation')
show_df(gross_margin_module['margin_input_view'], 4)
show_df(gross_margin_module['gross_margin_input_notes'].to_frame(), 4)

print('Gross margin module: outputs')
show_df(gross_margin_module['margin_view'], 4)
show_df(gross_margin_module['profit_view'], 4)


,Forecast Period,Auto GM,Energy GM,Software GM,Robotaxi GM (from launch onward),Total gross margin
0,1,0.1780,0.3062,0.0895,NaN,0.1852
1,2,0.1835,0.3153,0.1040,NaN,0.1937
2,3,0.1890,0.3244,0.1185,NaN,0.2022
3,4,0.1931,0.3335,0.1330,0.0500,0.2013
4,5,0.2000,0.3426,0.1475,0.0833,0.2112
5,6,0.2000,0.3518,0.1620,0.1167,0.2168
6,7,0.2000,0.3609,0.1765,0.1500,0.2236
7,8,0.2000,0.3700,0.1910,0.1833,0.2323
8,9,0.2000,0.3791,0.2055,0.2167,0.2428
9,10,0.2000,0.3800,0.2200,0.2500,0.2560


,Forecast Period,Auto GP,Energy GP,Software GP,Robotaxi GP,Total gross profit
0,1,15.4704,4.6977,1.3734,0.0000,21.5415
1,2,19.7761,5.8118,1.9172,0.0000,27.5051
2,3,25.0536,7.1841,2.6244,0.0000,34.8621
3,4,31.2325,8.8734,3.5387,0.5700,44.2146
4,5,38.8133,10.9518,4.7148,1.3145,55.7943
5,6,45.0234,13.5074,6.2211,2.5463,67.2982
6,7,50.4262,16.6482,8.1429,4.5299,79.7472
7,8,54.4603,20.5062,10.5865,7.6607,93.2137
8,9,58.8171,25.2430,13.6841,12.5271,110.2713
9,10,60.0000,30.4000,17.6000,20.0000,128.0000


### Step 4. Opex Module Run and Display

This section builds the operating expense model directly in Python.

Teaching focus:

- convert gross profit into EBIT by modeling operating expenses,
- understand how R&D, SG&A, and other expenses evolve over time,
- connect operating assumptions to FCFF inputs.


In [9]:
# Opex module run + display
opex_module = prepare_opex_module_package(core, rev_tbl_full, gross_margin_sheet_py)
operating_expense_sheet_py = opex_module['operating_expense']
l7_model['operating_expense'] = operating_expense_sheet_py.copy()

print('Opex module: key inputs used in the calculation')
show_df(opex_module['opex_input_view'].to_frame(), 4)
show_df(opex_module['rd_growth_input_view'], 4)

print('Opex module: outputs')
show_df(opex_module['opex_view'], 4)


,Forecast Period,R&D,R&D growth,SG&A,Restructuring and other,Total operating expense,Operating margin,Operating profit (EBIT)
0,1,8.7296,0.3617,5.834,0.494,15.0576,0.0557,6.4839
1,2,11.5134,0.3189,5.834,0.494,17.8414,0.0681,9.6636
2,3,14.7745,0.2832,5.834,0.494,21.1025,0.0798,13.7596
3,4,18.2618,0.2360,5.834,0.494,24.5898,0.0893,19.6248
4,5,21.8538,0.1967,5.834,0.494,28.1818,0.1045,27.6125
5,6,25.4359,0.1639,5.834,0.494,31.7639,0.1144,35.5343
6,7,28.9103,0.1366,5.834,0.494,35.2383,0.1248,44.5089
7,8,32.3795,0.1200,5.834,0.494,38.7075,0.1358,54.5062
8,9,35.7794,0.1050,5.834,0.494,42.1074,0.1501,68.1640
9,10,39.5362,0.1050,5.834,0.494,45.8642,0.1643,82.1358


### Step 5. Reinvestment and FCFF Module Run and Display

Teaching objective:

- connect operating margin assumptions to EBIT,
- connect revenue growth to reinvestment,
- compute FCFF and inspect implied capital-efficiency assumptions.

In [10]:
# Reinvestment / FCFF module run + display
reinvestment_fcff_module = prepare_reinvestment_fcff_module_package(core, rev_tbl_full, operating_expense_sheet_py)
reinvestment_tbl = reinvestment_fcff_module['reinvestment']
fcff_tbl = reinvestment_fcff_module['fcff']
l7_model.update({'reinvestment': reinvestment_tbl.copy(), 'fcff': fcff_tbl.copy()})

print('Reinvestment / FCFF module: key inputs used in the calculation')
show_df(reinvestment_fcff_module['reinvestment_input_view'], 4)
show_df(reinvestment_fcff_module['fcff_input_view'], 4)

print('Reinvestment / FCFF module: outputs')
show_df(reinvestment_tbl, 4)
show_df(reinvestment_fcff_module['fcff_view'], 4)


,Forecast Period,Sales to Capital,Revenue growth,Reinvestment
0,1,5.0,21.4967,4.2993
1,2,5.0,25.6810,5.1362
2,3,4.0,30.4449,7.6112
3,4,4.0,47.2076,11.8019
4,5,3.0,44.5299,14.8433
5,6,2.0,46.2954,23.1477
6,7,2.0,46.2163,23.1082
7,8,2.0,44.5240,22.2620
8,9,2.0,52.8822,26.4411
9,10,2.0,45.8921,22.9460


,Forecast Period,Total Revenues,Operating Margin,EBIT(1-t),Reinvestment,FCFF
0,1,116.3267,0.0557,4.9277,4.2993,0.6284
1,2,142.0076,0.0681,7.3444,5.1362,2.2082
2,3,172.4526,0.0798,10.4573,7.6112,2.8461
3,4,219.6601,0.0893,14.9149,11.8019,3.1130
4,5,264.1900,0.1045,20.9855,14.8433,6.1422
5,6,310.4854,0.1144,27.0061,23.1477,3.8584
6,7,356.7017,0.1248,33.8268,23.1082,10.7186
7,8,401.2258,0.1358,41.4247,22.2620,19.1627
8,9,454.1079,0.1501,51.8046,26.4411,25.3635
9,10,500.0000,0.1643,62.4232,22.9460,39.4772


## Results Display

### Step 6. Valuation and Sensitivity Module Run and Display

Teaching objective:

- estimate a **practice beta** from market data,
- compare slide assumptions vs code-derived values,
- understand why discount rate and terminal growth dominate valuation outcomes,
- learn how to switch between **default** and **custom** sensitivity grids from the Excel input sheet.

In [11]:
# Valuation / Sensitivity module run + display
valuation_module = prepare_valuation_sensitivity_module_package(
    core,
    fcff_tbl,
    dcf_pv_func=dcf_pv,
    terminal_value_func=terminal_value_growing_perpetuity,
    sensitivity_cfg=L7_SENSITIVITY_CONFIG,
    display_inputs=L7_DISPLAY_INPUTS,
    summary_core=L7_CORE_INPUTS,
    discount_rates_override=SENSITIVITY_DISCOUNT_RATES_OVERRIDE,
    growth_rates_override=SENSITIVITY_GROWTH_RATES_OVERRIDE,
)

valuation_from_core = valuation_module['valuation']
iv_from_core = valuation_module['iv_from_core']
sensitivity_discount_rates = valuation_module['sensitivity_discount_rates']
sensitivity_growth_rates = valuation_module['sensitivity_growth_rates']
sensitivity_matrix = valuation_module['sensitivity_matrix']
iv_matrix = valuation_module['iv_matrix']
l7_model['valuation'] = valuation_from_core

# Rebuild a one-package object for backward compatibility with later cells / optional exploration.
l7_run = {
    'core': L7_CORE_INPUTS,
    'constants': L7_CONSTANTS,
    'sensitivity_config': L7_SENSITIVITY_CONFIG,
    'ui': L7_UI,
    'extra_sections': L7_EXTRA_SECTIONS,
    'payload': l7_input_package.get('payload', {}),
    'input_table': L7_INPUT_TABLE,
    'model': l7_model,
    'sensitivity_matrix': sensitivity_matrix,
    'sensitivity_discount_rates': sensitivity_discount_rates,
    'sensitivity_growth_rates': sensitivity_growth_rates,
}

print('Valuation / Sensitivity module: key inputs used in the calculation')
show_df(valuation_module['valuation_input_view'].to_frame(), 4)
show_df(valuation_module['fcff_input_view'], 4)
show_df(valuation_module['sensitivity_info'].to_frame(), 4)
show_df(valuation_module['active_grids'], 4)

print('Valuation / Sensitivity module: outputs')
show_df(valuation_module['valuation_summary'].to_frame(), 4)
show_df(iv_from_core.to_frame('Intrinsic Value (USD bn)'), 4)
show_df(iv_matrix, 2)


,Value
Estimated beta (from Excel input sheet),2.0525
Latest local 10Y Treasury (from Excel input sheet),0.0418
ROE FY2025 (from Excel input sheet),0.0497
Tax rate (model input),0.2400
Forecast discount rate,0.0929
Terminal discount rate,0.0835
Terminal growth rate,0.0422


,Intrinsic Value (USD bn)
Terminal Value,996.2014
PV(Terminal Value),446.7432
PV(CF over Forecast Period),55.7831
Value of Operating Assets,502.5263


,Value
Use custom grid from Excel,0.000
Default discount delta (large),0.010
Default discount delta (small),0.005
Default growth delta,0.005
Notebook discount override active,0.000
Notebook growth override active,0.000


,Discount Rate Grid,Terminal Growth Grid
0,0.0735,0.0372
1,0.0785,0.0422
2,0.0835,0.0472
3,0.0885,NaN
4,0.0935,NaN


,3.72%,4.22%,4.72%
7.35%,619.17,710.93,837.58
7.85%,527.53,594.23,682.23
8.35%,456.27,506.43,570.40
8.85%,399.41,438.15,486.27
9.35%,353.08,383.65,420.83


### ESG / Policy Channel: ZEV Tradable Credits

This short step extracts the visible Tesla transfer rows from the slide screenshot to make the regulatory-credit mechanism more concrete.

In [12]:
# Tesla's visible ZEV credit transfers (from the teaching slide table)
tesla_visible_transfers = l7_read_csv('zev_visible_transfers_model_year_2023.csv')
show_df(tesla_visible_transfers, 0)
print('Visible Tesla-transfer credits total:', int(tesla_visible_transfers['Credits'].sum()))


,Transferee,Credits
0,Fiat Chrysler (Stellantis),69319
1,Ford,35000
2,Honda,7777
3,Mazda,30345
4,Mitsubishi,1288
5,Subaru,3376
6,Toyota,123414


Visible Tesla-transfer credits total: 270519


### Tradable Credits -> Ratios -> Intrinsic Value

Teaching objective:

- quantify the materiality of tradable-credit revenue,
- propagate the shock to profitability and liquidity ratios,
- compare intrinsic value **with vs without** tradable-credit revenue.

This section is the main bridge between **carbon/ESG policy context** and **valuation practice**.

In [13]:
# Tradable-credit scenario: ratios and intrinsic value impact (policy + ratio inputs loaded from the Excel input sheet)
credit_rev = float(L7_CONSTANTS['credit_revenue_usd'])
tc = L7_DISPLAY_INPUTS['tradable_credit_ratio_inputs_usd']

rev = float(tc['total_revenue_usd'])
ni_incl_nci = float(tc['net_income_including_noncontrolling_interests_usd'])
ni_common = float(tc['net_income_common_stockholders_usd'])

auto_base_year = int(tc.get('statement_base_year', 2025))

materiality = pd.Series({
    'Credit revenue (USD m)': credit_rev / 1e6,
    'Share of total revenue (%)': credit_rev / rev * 100,
    'Share of net income incl. NCI (%)': credit_rev / ni_incl_nci * 100,
    'Share of net income common (%)': credit_rev / ni_common * 100,
}, name='Value')
show_df(materiality.to_frame(), 2)

cash_ce = float(tc['cash_and_cash_equivalents_usd'])
retained = float(tc['retained_earnings_usd'])
ebit = float(tc['ebit_usd'])
scenario_tbl = pd.DataFrame([
    {
        'Credit retention': f'{r:.0%}',
        'Revenue (bn)': (rev - credit_rev * (1 - r)) / 1e9,
        'EBIT (bn)': (ebit - credit_rev * (1 - r)) / 1e9,
        'Net income common (bn)': (ni_common - credit_rev * (1 - r)) / 1e9,
        'Cash (bn)': (cash_ce - credit_rev * (1 - r)) / 1e9,
        'Retained earnings (bn)': (retained - credit_rev * (1 - r)) / 1e9,
    }
    for r in [1.0, 0.5, 0.0]
])
show_df(scenario_tbl, 2)

cogs = float(tc['cost_of_revenue_usd'])
gp = float(tc['gross_profit_usd'])
ca = float(tc['current_assets_usd'])
cl = float(tc['current_liabilities_usd'])
inv_cur = float(tc['inventory_current_usd'])
inv_prev = float(tc['inventory_previous_usd'])
ar_cur = float(tc['accounts_receivable_current_usd'])
ar_prev = float(tc['accounts_receivable_previous_usd'])
ap_cur = float(tc['accounts_payable_current_usd'])
ap_prev = float(tc['accounts_payable_previous_usd'])

inv_avg = (inv_cur + inv_prev) / 2
ap_avg = (ap_cur + ap_prev) / 2
ar_avg = (ar_cur + ar_prev) / 2
rev_after = rev - credit_rev

ratio_tbl = pd.DataFrame([
    ('Gross Margin', gp / rev, (gp - credit_rev) / rev_after),
    ('Operating Profit Margin', ebit / rev, (ebit - credit_rev) / rev_after),
    ('Net Profit Margin', ni_common / rev, (ni_common - credit_rev) / rev_after),
    ('Current Ratio', ca / cl, (ca - credit_rev) / cl),
    ('Quick Ratio', (ca - inv_cur) / cl, (ca - credit_rev - inv_cur) / cl),
    ('Cash Ratio', cash_ce / cl, (cash_ce - credit_rev) / cl),
    ('Inventory Turnover', cogs / inv_avg, cogs / inv_avg),
    ('A/P Turnover', cogs / ap_avg, cogs / ap_avg),
    ('A/R Turnover', rev / ar_avg, rev_after / ar_avg),
], columns=['Metric', 'Before', 'After']).set_index('Metric')
show_df(ratio_tbl, 2)

base_auto_with = base_rev['Automotive']
base_auto_without = base_auto_with - float(L7_CONSTANTS['credit_revenue_for_forecast_billion'])
auto_cagr_compare = pd.Series({
    f'Automotive CAGR with credit (base {auto_base_year})': (300 / base_auto_with) ** (1 / 10) - 1,
    f'Automotive CAGR without credit (base {auto_base_year})': (300 / base_auto_without) ** (1 / 10) - 1,
}, name='Implied CAGR')
show_df(auto_cagr_compare.to_frame(), 4)

iv_compare = l7_read_csv('valuation_without_credit_intrinsic_value_comparison.csv').set_index('Metric')
iv_compare['Change'] = iv_compare['Without credit'] - iv_compare['With credit']
iv_compare['Change (%)'] = iv_compare['Change'] / iv_compare['With credit'] * 100
show_df(iv_compare, 2)

,Value
Credit revenue (USD m),1993.00
Share of total revenue (%),2.10
Share of net income incl. NCI (%),51.70
Share of net income common (%),52.53


,Credit retention,Revenue (bn),EBIT (bn),Net income common (bn),Cash (bn),Retained earnings (bn)
0,100%,94.83,5.62,3.79,16.51,39.00
1,50%,93.83,4.62,2.80,15.52,38.01
2,0%,92.83,3.62,1.80,14.52,37.01


,Before,After
Metric,,
Gross Margin,0.18,0.16
Operating Profit Margin,0.06,0.04
Net Profit Margin,0.04,0.02
Current Ratio,2.16,2.10
Quick Ratio,1.77,1.71
Cash Ratio,0.52,0.46
Inventory Turnover,6.37,6.37
A/P Turnover,6.02,6.02
A/R Turnover,21.09,20.64


,Implied CAGR
Automotive CAGR with credit (base 2025),0.1574
Automotive CAGR without credit (base 2025),0.1607


,With credit,Without credit,Change,Change (%)
Metric,,,,
Terminal Value,996.20,825.86,-170.34,-17.10
PV(Terminal Value),446.74,370.35,-76.39,-17.10
PV(CF over forecast period),55.78,36.03,-19.75,-35.41
Value of operating assets,502.53,406.38,-96.15,-19.13


### Lesson Summary

In this lesson, you built a valuation workflow directly in Python with a **single Excel input sheet**.

What you practiced:

1. Updating a fixed-format input sheet (one place to edit assumptions)
2. Running a reusable valuation function that can be adapted to other companies
3. Building revenue, margin, operating expense, reinvestment, and FCFF forecasts automatically
4. Estimating intrinsic value with DCF
5. Running sensitivity analysis with default or custom discount-rate / growth-rate grids
6. Tracing ESG / policy impacts (tradable credits) into ratios and valuation

A strong modeling habit is to keep the workflow in this order:

- input sheet,
- model construction,
- result display,
- scenario analysis.

When next year's Tesla data is available, students can update the Excel input sheet and rerun the notebook to regenerate the valuation outputs.